This notebook refer the public notebook shared by @seshurajup


In [ ]:
import os
import json
import zipfile
from tqdm import tqdm
from rich import print
from collections import defaultdict
import copy

def check_code_solution(solution_code, task_data, validate_all=False):
    """
    Validate solution code against task examples
    validate_all: if True, validate against ALL examples, not just first 3
    """
    try:
        # Create isolated namespace to avoid pollution
        namespace = {}
        exec(solution_code, namespace)
        
        # Check if function 'p' exists
        if 'p' not in namespace:
            return False
        
        # Get all examples
        all_examples = task_data['train'] + task_data['test'] + task_data['arc-gen']
        
        # Determine how many examples to check
        examples_to_check = all_examples if validate_all else all_examples[:3]
        
        for example in examples_to_check:
            # Deep copy to avoid mutation
            input_grid = copy.deepcopy(example['input'])
            expected = example['output']
            
            try:
                actual = namespace['p'](input_grid)
                if actual != expected:
                    return False
            except:
                return False
                
        return True
    except Exception as e:
        return False

def get_bytes(code: str) -> int:
    """Calculate byte count for scoring"""
    return len(code.encode('utf-8'))

def calculate_score(bytes_count, is_correct):
    """Calculate score based on competition rules"""
    if is_correct:
        return max(1, 2500 - bytes_count)
    else:
        return 0.001

# All submission paths to check
submissions = [
    "/kaggle/input/solved-127-problems-local-pleaseupvote",
    "/kaggle/input/111-solved-stater-neurips-2025-google-code-golf,"
    "/kaggle/input/a-bit-more-of-code-golf-ast-regex-and-more",
    "/kaggle/input/stater-neurips-2025-google-code-golf",
    "/kaggle/input/google-code-golf-arc-solver-lb",
    "/kaggle/input/lb47268-neurips-2025-google-code-golf",
    "/kaggle/input/qwen2-5-32b",
    "/kaggle/input/local-score-liah",
    "/kaggle/input/google-code-golf-ensemble-local-score",
    "/kaggle/input/qwen2-5-32b-arc-local-score-added-raster-code",
    "/kaggle/input/32-solved-yay-2d-1d-raster-compression-expansion",
    "/kaggle/input/qwen2-5-32b-arc-local-score",
    "/kaggle/input/solved-48-problems-by-resemble-all-pub-curnotebook",
    "/kaggle/input/oh-barnacles"
]

# Fallback solution
simple_solution = """def p(g):
    return g"""

# Initialize counters
solved = 0
total_score = 0
source_stats = defaultdict(int)  # Track which submission provided best solutions
validation_mode = "quick"  # "quick" for first 3 examples, "full" for all examples

# Create output directory
os.makedirs("/kaggle/working/submission", exist_ok=True)

print(f"[yellow]Starting ensemble with {len(submissions)} submission sources[/yellow]")
print(f"[yellow]Validation mode: {validation_mode}[/yellow]\n")

# Process each task
for task_num in tqdm(range(1, 401), desc="Processing tasks"):
    task_id = f"{task_num:03d}"
    
    # Load task data
    task_data_path = f"/kaggle/input/google-code-golf-2025/task{task_id}.json"
    task_data = json.load(open(task_data_path))
    
    # Collect all valid solutions for this task
    valid_solutions = []
    
    for submission_path in submissions:
        task_code_path = f"{submission_path}/submission/task{task_id}.py"
        
        if not os.path.exists(task_code_path):
            continue
            
        try:
            with open(task_code_path, 'r') as f:
                solution_code = f.read()
            
            # Quick validation (first 3 examples)
            if check_code_solution(solution_code, task_data, validate_all=False):
                bytes_count = get_bytes(solution_code)
                valid_solutions.append({
                    "code": solution_code,
                    "bytes": bytes_count,
                    "source": submission_path.split('/')[-1],
                    "score": calculate_score(bytes_count, True)
                })
        except Exception as e:
            # Skip problematic files
            continue
    
    # Select best solution (minimize bytes = maximize score)
    if valid_solutions:
        # Sort by bytes (ascending) to maximize score
        valid_solutions.sort(key=lambda x: x['bytes'])
        best_solution = valid_solutions[0]
        
        # Optional: Double-check the best solution with full validation
        if validation_mode == "full":
            if not check_code_solution(best_solution['code'], task_data, validate_all=True):
                # Try next best solution
                for sol in valid_solutions[1:]:
                    if check_code_solution(sol['code'], task_data, validate_all=True):
                        best_solution = sol
                        break
                else:
                    # No fully valid solution found
                    best_solution = None
        
        if best_solution:
            final_code = best_solution['code']
            final_bytes = best_solution['bytes']
            score = best_solution['score']
            source_stats[best_solution['source']] += 1
            solved += 1
            
            # Show details for solved tasks
            print(f"[green]{task_id} - solved ({final_bytes} bytes, score: {score:.1f}, "
                  f"from: {best_solution['source']})[/green]")
        else:
            # Fallback if no valid solution
            final_code = simple_solution
            final_bytes = get_bytes(simple_solution)
            score = 0.001
    else:
        # No valid solutions found
        final_code = simple_solution
        final_bytes = get_bytes(simple_solution)
        score = 0.001
    
    total_score += score
    
    # Write the best solution
    with open(f"/kaggle/working/submission/task{task_id}.py", "w") as f:
        f.write(final_code)

# Create submission zip file
print("\n[yellow]Creating submission.zip...[/yellow]")
with zipfile.ZipFile("/kaggle/working/submission.zip", "w") as zipf:
    for task_num in range(1, 401):
        task_id = f"{task_num:03d}"
        zipf.write(f"/kaggle/working/submission/task{task_id}.py", 
                   arcname=f"task{task_id}.py")

# Print final statistics
print(f"\n[green]{'='*50}[/green]")
print(f"[green]FINAL RESULTS:[/green]")
print(f"[green]Total solved: {solved} / 400[/green]")
print(f"[blue]LB Score: {total_score:.3f}[/blue]")
print(f"[yellow]Average bytes per solved task: {sum(get_bytes(open(f'/kaggle/working/submission/task{i:03d}.py').read()) for i in range(1, 401) if os.path.exists(f'/kaggle/working/submission/task{i:03d}.py')) / max(1, solved):.1f}[/yellow]")

# Show contribution from each source
if source_stats:
    print(f"\n[cyan]Solution sources breakdown:[/cyan]")
    for source, count in sorted(source_stats.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / solved * 100) if solved > 0 else 0
        print(f"  {source}: {count} solutions ({percentage:.1f}%)")
        
print(f"\n[green]{'='*50}[/green]")